In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import copy

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

# %load_ext autoreload
# %autoreload 2

In [2]:
mydateparser = lambda x: datetime.strptime(x, "%Y-%m-%d")
info = pd.read_csv('../data/snp_info.csv', index_col=0)

In [3]:
stocks_analysis = ['KO', 'TGT', 'PFE', 'MSFT', 'CVX', 'DVN', 'DAL', 'JPM', 'PEP', 'AAPL']

In [4]:
stocks = pd.read_csv('stocks_by_sector.csv')
stocks

,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology
0,T,F,KO,MRO,BAC,PFE,GE,AAPL
1,TWTR,GM,KR,KMI,WFC,MRK,DAL,AMD
2,FB,EBAY,PG,XOM,C,GILD,CSX,MU
3,CMCSA,SBUX,WMT,HAL,RF,BMY,AAL,MSFT
4,VZ,NKE,MDLZ,WMB,JPM,BSX,LUV,INTC
5,NFLX,M,MO,COP,KEY,ABT,FAST,CSCO
6,DIS,MGM,COTY,SLB,MS,CVS,CAT,HPE
7,ATVI,TJX,WBA,DVN,HBAN,ABBV,JCI,ORCL
8,IPG,TGT,PM,CVX,SCHW,JNJ,UAL,NVDA
9,DISCA,NWL,PEP,COG,SYF,MDT,UNP,AMAT


In [5]:
idx_dict = {}
for i in range(len(stocks.index)):
    for j in range(len(stocks.columns)):
        stock = stocks.iloc[i,j]
        idx_dict[stock] = (i,j)

In [6]:
import itertools

all_stocks = []
for i in range(len(stocks.columns)):
    all_stocks.append(list(stocks.iloc[:,i]))
all_stocks = list(itertools.chain.from_iterable(all_stocks))

---------
Get Samples and Labels
---------

In [7]:
samples = pd.read_csv('samples_sector.csv', index_col=0, header=[0,1,2], parse_dates=True, date_parser=mydateparser)
samples.head()

Sector     Communication Services                                             \
Asset                     asset_1                                              
Metrics                     Close        RSI  k_percent  r_percent      MACD   
Date                                                                           
2016-01-22              35.139999  66.984998  97.740062  -2.259938  0.092199   
2016-01-25              35.000000  63.461411  78.712861 -21.287139  0.123844   
2016-01-26              35.400002  68.861369  92.129714  -7.870286  0.170558   
2016-01-27              35.480000  69.888295  86.610888 -13.389112  0.209019   
2016-01-28              35.529999  70.587814  88.702907 -11.297093  0.239060   

Sector                                                                  \
Asset                                                          asset_2   
Metrics    MACD_EMA9 Price Rate Of Change On Balance Volume      Close   
Date                                                                     
2016-01-22  0.021266             0.047704          10592300  17.840000   
2016-01-25  0.042529             0.030928         -19672100  17.020000   
2016-01-26  0.068877             0.044248          22005000  17.010000   
2016-01-27  0.097551             0.051571          72268000  16.780001   
2016-01-28  0.126372             0.035860          98452400  16.490000   

Sector                       ...        Information Technology  \
Asset                        ...                       asset_9   
Metrics           RSI        ...          Price Rate Of Change   
Date                         ...                                 
2016-01-22  30.079617        ...                     -0.039824   
2016-01-25  23.781919        ...                     -0.042453   
2016-01-26  23.712060        ...                     -0.049039   
2016-01-27  21.997274        ...                     -0.030759   
2016-01-28  19.903236        ...                     -0.021625   

Sector                                                                    \
Asset                          asset_10                                    
Metrics    On Balance Volume      Close        RSI  k_percent  r_percent   
Date                                                                       
2016-01-22          -9242700  17.049999  51.119297  43.165445 -56.834555   
2016-01-25         -16010900  16.910000  47.801520  38.129494 -61.870506   
2016-01-26         -10312300  17.350000  57.746426  63.025232 -36.974768   
2016-01-27         -16104800  17.090000  51.107526  70.454523 -29.545477   
2016-01-28         -23024200  16.680000  42.266121  47.976887 -52.023113   

Sector                                                                 
Asset                                                                  
Metrics         MACD MACD_EMA9 Price Rate Of Change On Balance Volume  
Date                                                                   
2016-01-22 -0.097708 -0.124555             0.015485         -82509700  
2016-01-25 -0.078060 -0.114917             0.013789         -91721800  
2016-01-26 -0.035484 -0.098570             0.035821         -75935100  
2016-01-27 -0.018663 -0.082221             0.054938         -88525300  
2016-01-28 -0.031403 -0.071871            -0.007733        -106022400  

[5 rows x 640 columns]

In [8]:
labels = pd.read_csv('labels_sector.csv', index_col='Date', parse_dates=True, date_parser=mydateparser)
labels.head()

,T,TWTR,FB,CMCSA,VZ,NFLX,DIS,ATVI,IPG,DISCA,...,AAPL,AMD,MU,MSFT,INTC,CSCO,HPE,ORCL,NVDA,AMAT
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-22,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
2016-01-25,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2016-01-26,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2016-01-27,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2016-01-28,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


------------------------------------
Training and Prediction (Base)
------------------------------------

In [47]:
from GRTEL.classification import MultiClassifier
from GRTEL.utils import downturn_confidence, print_scores, print_1_percentage, confusion_matrix_metrics 

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [48]:
X = samples
y = np.array(labels)


# Split into train and test set
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

test_size = 0.25
k = int(len(X) * (1. - test_size))

X_train, X_test = X[:k], X[k:]
y_train, y_test = y[:k], y[k:]

In [49]:
# Initialise classifier
n_classes = y.shape[1]

multi_clf = MultiClassifier(n_classes)

# Train classifer
multi_clf.fit(X_train, y_train)

0 - 1 - 2 - 3 - 4 - 5 - 6 - 7 - 8 - 9 - 10 - 11 - 12 - 13 - 14 - 15 - 16 - 17 - 18 - 19 - 20 - 21 - 22 - 23 - 24 - 25 - 26 - 27 - 28 - 29 - 30 - 31 - 32 - 33 - 34 - 35 - 36 - 37 - 38 - 39 - 40 - 41 - 42 - 43 - 44 - 45 - 46 - 47 - 48 - 49 - 50 - 51 - 52 - 53 - 54 - 55 - 56 - 57 - 58 - 59 - 60 - 61 - 62 - 63 - 64 - 65 - 66 - 67 - 68 - 69 - 70 - 71 - 72 - 73 - 74 - 75 - 76 - 77 - 78 - 79 - 


In [50]:
# train accuracy
train_score = multi_clf.score(X_train, y_train)
train_accuracy_df = pd.DataFrame(np.array(train_score).reshape(stocks.shape[1], stocks.shape[0])).T
train_accuracy_df.columns = stocks.columns

# test accuracy
test_score = multi_clf.score(X_test, y_test)
test_accuracy_df = pd.DataFrame(np.array(test_score).reshape(stocks.shape[1], stocks.shape[0])).T
test_accuracy_df.columns = stocks.columns

# confusion matrices
conf_matrices = multi_clf.confusion_matrices(X_test, y_test)
conf_matrices = np.array(conf_matrices).reshape(stocks.shape[1], stocks.shape[0], 2, 2)



## Print performance results
results = pd.DataFrame(columns=all_stocks, index=['train acc', 'test acc', 'precision', 'recall', 'downturn_precision', 'specificity'], dtype=float)
for stock in all_stocks:
    idx = idx_dict[stock]
    results[stock]['train acc'] = train_accuracy_df.iloc[idx]
    results[stock]['test acc'] = test_accuracy_df.iloc[idx]
    
    conf_matrix = conf_matrices[idx[1],idx[0]]
    accuracy, precision, recall, specificity, downturn_precision = confusion_matrix_metrics(conf_matrix)
    
    results[stock]['precision'] = precision
    results[stock]['recall'] = recall
    results[stock]['downturn_precision'] = downturn_precision
    results[stock]['specificity'] = specificity
    
results

,T,TWTR,FB,CMCSA,VZ,NFLX,DIS,ATVI,IPG,DISCA,...,AAPL,AMD,MU,MSFT,INTC,CSCO,HPE,ORCL,NVDA,AMAT
train acc,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
test acc,0.524590,0.540984,0.442623,0.486339,0.557377,0.497268,0.393443,0.497268,0.497268,0.513661,...,0.513661,0.398907,0.568306,0.437158,0.562842,0.612022,0.508197,0.573770,0.535519,0.404372
precision,0.487395,0.580000,0.471429,0.632653,0.632812,0.500000,0.376812,0.481250,0.500000,0.471014,...,0.692308,0.928571,0.367347,0.540000,0.573913,0.676768,0.428571,0.696429,0.492754,0.250000
recall,0.690476,0.580000,0.336735,0.289720,0.704348,0.565217,0.276596,0.895349,0.673913,0.802469,...,0.095745,0.106557,0.272727,0.486486,0.680412,0.632075,0.139535,0.390000,0.404762,0.313433
downturn_precision,0.593750,0.493976,0.424779,0.432836,0.381818,0.493671,0.403509,0.608696,0.491525,0.644444,...,0.500000,0.355030,0.641791,0.313253,0.544118,0.535714,0.522581,0.519685,0.561404,0.535354
specificity,0.383838,0.493976,0.564706,0.763158,0.308824,0.428571,0.516854,0.144330,0.318681,0.284314,...,0.955056,0.983607,0.735043,0.361111,0.430233,0.584416,0.835052,0.795181,0.646465,0.456897


In [51]:
final_results = results[stocks_analysis].copy()
final_results['Average'] = final_results.mean(axis=1)
final_results['Overall'] = results.mean(axis=1)
final_results

,KO,TGT,PFE,MSFT,CVX,DVN,DAL,JPM,PEP,AAPL,Average,Overall
train acc,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
test acc,0.513661,0.442623,0.475410,0.437158,0.546448,0.508197,0.530055,0.480874,0.557377,0.513661,0.500546,0.510109
precision,0.630769,0.482270,0.658537,0.540000,0.534884,0.475610,0.600000,0.412281,0.566667,0.692308,0.559332,0.528656
recall,0.386792,0.701031,0.442623,0.486486,0.750000,0.453488,0.588785,0.626667,0.841584,0.095745,0.537320,0.477433
downturn_precision,0.449153,0.309524,0.326733,0.313253,0.574074,0.534653,0.435897,0.594203,0.515152,0.500000,0.455264,0.506208
specificity,0.688312,0.151163,0.540984,0.361111,0.340659,0.556701,0.447368,0.379630,0.207317,0.955056,0.462830,0.546827


In [52]:
# #Scores
# score = multi_clf.score(X_train, y_train)
# print("\nClassification accuracy (Train):")
# print_scores(score); print()

# score = multi_clf.score(X_test, y_test)
# print("Classification accuracy (Test):")
# print_scores(score); print()

# print("Percentage of 1s (Test):")
# print_1_percentage(y_test, n_classes); print()